In [42]:
import pandas as pd
import numpy as np

df0 = pd.read_csv('msnbc990928.seq.gz', compression = 'gzip')

df0

,% Different categories found in input file:
0,frontpage news tech local opinion on-air misc ...
1,% Sequences:
2,1 1
3,2
4,3 2 2 4 2 2 2 3 3
...,...
989815,9 9 9 9 9 9 9 9
989816,1 2
989817,1 1 1 1
989818,1 1 1 1 1 1 1


In [43]:
#first row contains header info
#need to re-do header and column name, and reset index

#Rename column
df0.columns = ['%Sequences']

#Drop first two rows
df = df0.drop([df0.index[0], df0.index[1]])

df.reset_index(level=df.index.names, inplace=True)

df_save = df

In [44]:
#total visits per user

df['total_visits'] = df['%Sequences'].str.count(' ') 

In [45]:
# user navigation path in the list 'Browse'

df['browse'] = df['%Sequences'].str.split(' ')

# remove the last element ' '
for i in df['browse']:
    i.pop()

In [46]:
# entry site: web site user visited first

df.loc[:, 'entry'] = df['browse'].map(lambda x: x[0])

In [47]:
#number of visits per page per user

from collections import Counter

df.loc[:, 'sep_count'] = df['browse'].map(lambda x: dict(Counter(x)).items())

In [48]:
df

,index,%Sequences,total_visits,browse,entry,sep_count
0,2,1 1,2,"[1, 1]",1,"((1, 2))"
1,3,2,1,[2],2,"((2, 1))"
2,4,3 2 2 4 2 2 2 3 3,9,"[3, 2, 2, 4, 2, 2, 2, 3, 3]",3,"((3, 3), (2, 5), (4, 1))"
3,5,5,1,[5],5,"((5, 1))"
4,6,1,1,[1],1,"((1, 1))"
...,...,...,...,...,...,...
989813,989815,9 9 9 9 9 9 9 9,8,"[9, 9, 9, 9, 9, 9, 9, 9]",9,"((9, 8))"
989814,989816,1 2,2,"[1, 2]",1,"((1, 1), (2, 1))"
989815,989817,1 1 1 1,4,"[1, 1, 1, 1]",1,"((1, 4))"
989816,989818,1 1 1 1 1 1 1,7,"[1, 1, 1, 1, 1, 1, 1]",1,"((1, 7))"


In [58]:
#table to show visits per page/user

df_0 = df.copy(deep=True)[['browse']]
df_0_exploded = df_0.explode('browse')
df_0_exploded['browse'] = df_0_exploded['browse'].astype(int)
df_0_exploded = df_0_exploded.reset_index()
df_0_pivot = df_0_exploded.groupby(['index', 'browse'])['browse'].count().unstack(level=-1)

#Filled NA by 0
df_0_pivot = df_0_pivot.fillna(0).astype('int64')
df_0_pivot = df_0_pivot.sort_index(axis=1)
    
df_0_pivot

browse,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
index,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,5,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989813,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0
989814,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
989815,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
#parse header from the first row of the raw data df0

category_listing = df0['%Sequences'].iloc[0].split(' ')
lookup = dict(zip([str(i) for i in range(1, len(category_listing)+1)], category_listing))

df_0_pivot.columns = list(lookup.values())
df_visits = df_0_pivot
df_visits['user_id'] = df_visits.index + 2

df_visits

,frontpage,news,tech,local,opinion,on-air,misc,weather,msn-news,health,living,business,msn-sports,sports,summary,bbs,travel,user_id
index,,,,,,,,,,,,,,,,,,
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
2,0,5,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989813,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,989815
989814,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,989816
989815,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,989817


In [60]:
#create summary table for total visits per site
#calculate node weights

df3 = pd.DataFrame(df_0_pivot.sum(axis=0))
df3['site_description'] = df3.index
df3.columns= ['site_visit_counts', 'site_description']

lookup_str = dict(zip([str(i) for i in range(1, len(category_listing)+1)], [[c] for c in category_listing]))
df_lookup = pd.DataFrame.from_dict(lookup_str).transpose().reset_index()
df_lookup.columns = ['site_id', 'site_description']

df_category_freq = df_lookup.merge(df3, on='site_description')

df_category_freq

,site_id,site_description,site_visit_counts
0,1,frontpage,940469
1,2,news,452387
2,3,tech,207479
3,4,local,386217
4,5,opinion,151409
5,6,on-air,414928
6,7,misc,305615
7,8,weather,439398
8,9,msn-news,196614
9,10,health,131760


In [61]:
# table to show edge paths and edge weights
# explode %Sequences strings 

test_df = df.copy(deep = True)[['index', '%Sequences']]
test_df['site_id'] = test_df['%Sequences'].apply(lambda x:x.strip(' ').split(' '))
exploded_df = test_df.explode('site_id')

exploded_df

,index,%Sequences,site_id
0,2,1 1,1
0,2,1 1,1
1,3,2,2
2,4,3 2 2 4 2 2 2 3 3,3
2,4,3 2 2 4 2 2 2 3 3,2
...,...,...,...
989816,989818,1 1 1 1 1 1 1,1
989816,989818,1 1 1 1 1 1 1,1
989816,989818,1 1 1 1 1 1 1,1
989816,989818,1 1 1 1 1 1 1,1


In [62]:
# calculate edge paths

exploded_df = exploded_df.merge(df_lookup, on = 'site_id')
exploded_df['next_sequence_desc'] = exploded_df.groupby(['index'])\
['site_description'].shift(-1)

exploded_df['next_sequence'] = exploded_df.groupby(['index'])\
['site_id'].shift(-1)

exploded_df['edge_path'] = list(zip(exploded_df.site_id, exploded_df.next_sequence))
exploded_df['edge_path_desc'] = list(zip(exploded_df.site_description, exploded_df.next_sequence_desc))

In [63]:
exploded_df

,index,%Sequences,site_id,site_description,next_sequence_desc,next_sequence,edge_path,edge_path_desc
0,2,1 1,1,frontpage,frontpage,1,"(1, 1)","(frontpage, frontpage)"
1,2,1 1,1,frontpage,NaN,NaN,"(1, nan)","(frontpage, nan)"
2,6,1,1,frontpage,NaN,NaN,"(1, nan)","(frontpage, nan)"
3,8,1 1,1,frontpage,frontpage,1,"(1, 1)","(frontpage, frontpage)"
4,8,1 1,1,frontpage,NaN,NaN,"(1, nan)","(frontpage, nan)"
...,...,...,...,...,...,...,...,...
4698789,989593,17,17,travel,NaN,NaN,"(17, nan)","(travel, nan)"
4698790,989603,1 17 17 17 1,17,travel,travel,17,"(17, 17)","(travel, travel)"
4698791,989603,1 17 17 17 1,17,travel,travel,17,"(17, 17)","(travel, travel)"
4698792,989603,1 17 17 17 1,17,travel,NaN,NaN,"(17, nan)","(travel, nan)"


In [64]:
# calculate edge weights

weights_df =exploded_df['edge_path_desc'].value_counts().reset_index()
weights_df.columns = ['edge_path', 'edge_path_counts']

weights_df

,edge_path,edge_path_counts
0,"(frontpage, frontpage)",627288
1,"(weather, weather)",343783
2,"(news, news)",277101
3,"(sports, sports)",276742
4,"(local, local)",264498
...,...,...
165,"(msn-news, bbs)",31
166,"(tech, bbs)",29
167,"(local, bbs)",28
168,"(msn-sports, travel)",27


In [65]:
#calculate normalized edge weights

weights_df_norm = weights_df
col_total = weights_df_norm['edge_path_counts'].sum()
weights_df_norm['normalized_edge_path']= weights_df_norm['edge_path_counts']\
.apply(lambda x: round(float(x)/col_total , 10)) 

weights_df_norm.columns = ['edge_path', 'edge_path_counts', 'normalized_edge_path']

In [66]:
weights_df_norm

,edge_path,edge_path_counts,normalized_edge_path
0,"(frontpage, frontpage)",627288,0.133500
1,"(weather, weather)",343783,0.073164
2,"(news, news)",277101,0.058973
3,"(sports, sports)",276742,0.058896
4,"(local, local)",264498,0.056291
...,...,...,...
165,"(msn-news, bbs)",31,0.000007
166,"(tech, bbs)",29,0.000006
167,"(local, bbs)",28,0.000006
168,"(msn-sports, travel)",27,0.000006


In [67]:
#entry site frequency: first site user visited

df_entry_freq = df['entry'].value_counts().reset_index()
df_entry_freq.columns = ['site_id', 'entry_counts']
df_entry_freq = df_entry_freq.merge(df_lookup, on = 'site_id')
df_entry_freq.columns = ['site_id', 'entry_counts', 'site_description']

df_entry_freq

,site_id,entry_counts,site_description
0,1,277376,frontpage
1,6,163077,on-air
2,2,77138,news
3,8,71828,weather
4,3,66554,tech
5,9,65278,msn-news
6,13,62986,msn-sports
7,12,56742,business
8,14,53639,sports
9,4,51040,local


In [70]:
df_save.to_csv('MSNBC Data original.csv', index = False, header=True)
df_visits.to_csv('MSNBC visits per page per user.csv', index = False, header=True)
df_category_freq.to_csv('MSNBC visits per page.csv', index = False, header=True)
exploded_df.to_csv('MSNBC network edge weights calc.csv', index = False, header=True)
df_entry_freq.to_csv('MSNBC first visited pages per user.csv', index = False, header=True)
weights_df.to_csv('MSNBC network edge weights.csv', index = False, header=True)